In [168]:
#Source: https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

# For sending GET requests from the API
#import requests
import pip._vendor.requests 
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
from dotenv import load_dotenv

In [169]:
load_dotenv()

def auth():
    return os.environ.get("bearer-token")

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [170]:

def create_url(userid, max_results):
    
    #search_url = "https://api.twitter.com/2/users/:id/tweets" #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/users/{}/tweets".format(userid)
    

    #change params based on the endpoint you are using
    query_params = {'id': userid,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',                
                    'tweet.fields': 'text,author_id,in_reply_to_user_id,geo,created_at,public_metrics,referenced_tweets',
                    #'user.fields': 'name,username',
                    #'place.fields': 'country',
                    'next_token': {}}
    return (search_url, query_params)

In [171]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = pip._vendor.requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [172]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
user_id = "1113874649824354304"
max_results = 15

In [173]:
url = create_url(user_id, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 400


Exception: (400, '{"errors":[{"parameters":{"id":["1113874649824354304","1113874649824354304"]},"message":"Duplicate parameters are not allowed: the `id` query parameter"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')

In [ ]:
df = pd.DataFrame(json_response['data'])
df.to_csv('hansTweetData.csv')